<a href="https://colab.research.google.com/github/ChallapalliHemanthsaikumar/spark_sql/blob/main/Spark_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null


In [2]:
!wget -q https://archive.apache.org/dist/spark/spark-3.2.0/spark-3.2.0-bin-hadoop3.2.tgz
!tar xf spark-3.2.0-bin-hadoop3.2.tgz

In [3]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.0-bin-hadoop3.2"

In [5]:
!pip install -q findspark

In [7]:
# import spark
import findspark
findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName("Colab") \
    .getOrCreate()

In [8]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv

--2024-05-16 21:50:30--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMSkillsNetwork-BD0231EN-Coursera/datasets/NASA_airfoil_noise_raw.csv
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.63.118.104
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.63.118.104|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 60682 (59K) [text/csv]
Saving to: ‘NASA_airfoil_noise_raw.csv’

NASA_airfoil_noise_ 100%[===================>]  59.26K   237KB/s    in 0.3s    

2024-05-16 21:50:31 (237 KB/s) - ‘NASA_airfoil_noise_raw.csv’ saved [60682/60682]



In [25]:
!ls


NASA_airfoil_noise_raw.csv  sample_data  spark-3.2.0-bin-hadoop3.2  spark-3.2.0-bin-hadoop3.2.tgz


In [26]:
df = spark.read.csv('NASA_airfoil_noise_raw.csv', header=True, inferSchema=True)

In [27]:
df.show(10)

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|      800|          0.0|     0.3048|              71.3|             0.00266337|   126.201|
|     1000|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     1250|          0.0|     0.3048|              71.3|             0.00266337|   125.951|
|     1600|          0.0|     0.3048|              71.3|             0.00266337|   127.591|
|     2000|          0.0|     0.3048|              71.3|             0.00266337|   127.461|
|     2500|          0.0|     0.3048|              71.3|             0.00266337|   125.571|
|     3150|          0.0|     0.3048|              71.3|             0.00266337|   125.201|
|     4000|          0.0|     0.3048|              71.3|             0.00266337|

In [28]:
df.count()

1522

In [29]:
df = df.dropDuplicates()
df.show()

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|   115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|   121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|   115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|   131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|   131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|   122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|   129.116|
|     1600|          4.8|     0.0254|              39.6|             9.07475E-4|

Print the total number of rows in the dataset

In [30]:
df.count()

1503

In [31]:
df = df.dropna()
df.count()
df.show()

+---------+-------------+-----------+------------------+-----------------------+----------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevel|
+---------+-------------+-----------+------------------+-----------------------+----------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|   115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|   121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|   115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|   131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|   131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|   122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|   129.116|
|     1600|          4.8|     0.0254|              39.6|             9.07475E-4|

In [32]:
df.printSchema()

root
 |-- Frequency: integer (nullable = true)
 |-- AngleOfAttack: double (nullable = true)
 |-- ChordLength: double (nullable = true)
 |-- FreeStreamVelocity: double (nullable = true)
 |-- SuctionSideDisplacement: double (nullable = true)
 |-- SoundLevel: double (nullable = true)



In [33]:
df = df.withColumnRenamed('SoundLevel','SoundLevelDecibels')

In [34]:
df.write.parquet('NASA_airfoil_noise_cleaned.parquet')

In [35]:
df.write.option("compression", "snappy") \
    .mode("overwrite") \
    .parquet("NASA_airfoil_noise_cleaned.parquet")

### Loading Data For Spark ML

In [112]:
df = spark.read.parquet('NASA_airfoil_noise_cleaned.parquet')

In [113]:

rawcount4 = df.count()
print(rawcount4)
df.printSchema()

1499
root
 |-- Frequency: integer (nullable = true)
 |-- AngleOfAttack: double (nullable = true)
 |-- ChordLength: double (nullable = true)
 |-- FreeStreamVelocity: double (nullable = true)
 |-- SuctionSideDisplacement: double (nullable = true)
 |-- SoundLevelDecibels: double (nullable = true)



In [114]:
# from sklearn import VectorAssembler

from pyspark.ml.feature import VectorAssembler
assembler = VectorAssembler(inputCols=['Frequency', 'AngleOfAttack', 'ChordLength', 'FreeStreamVelocity', 'SuctionSideDisplacement'], outputCol='features')
# df = assembler.transform(df)

In [115]:
df.printSchema()

root
 |-- Frequency: integer (nullable = true)
 |-- AngleOfAttack: double (nullable = true)
 |-- ChordLength: double (nullable = true)
 |-- FreeStreamVelocity: double (nullable = true)
 |-- SuctionSideDisplacement: double (nullable = true)
 |-- SoundLevelDecibels: double (nullable = true)



In [116]:
df.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|           115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|           121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|           115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|           131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|           131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|           122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|           129.116|


In [117]:
from pyspark.ml.feature import StandardScaler
scaler = StandardScaler(inputCol="features", outputCol="scaledFeatures", withStd=True, withMean=True)

In [124]:
from pyspark.ml.regression import LinearRegression
# lr = LinearRegression(featuresCol='scaledFeatures',predictionCol='SoundLevelDecibels')
lr = LinearRegression(featuresCol="scaledFeatures", labelCol="SoundLevelDecibels")

In [125]:
from pyspark.ml import Pipeline
# import pyspark.ml.
pipeline = Pipeline(stages=[assembler, scaler, lr])

In [126]:
seed = 42
# df.randomSplit([0.7, 0.3], seed)
(training_data,testing_data) = df.randomSplit([0.7, 0.3], seed)

In [127]:
df.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|     4000|          3.0|     0.3048|              31.7|             0.00529514|           115.608|
|     3150|          2.0|     0.2286|              31.7|             0.00372371|           121.527|
|     2000|          7.3|     0.2286|              31.7|              0.0132672|           115.309|
|     2000|          5.4|     0.1524|              71.3|             0.00401199|           131.111|
|      500|          9.9|     0.1524|              71.3|              0.0193001|           131.279|
|     1000|         12.6|     0.1524|              71.3|              0.0483159|           122.044|
|    12500|          0.0|     0.0254|              39.6|             4.28464E-4|           129.116|


In [128]:
training_data.show()

+---------+-------------+-----------+------------------+-----------------------+------------------+
|Frequency|AngleOfAttack|ChordLength|FreeStreamVelocity|SuctionSideDisplacement|SoundLevelDecibels|
+---------+-------------+-----------+------------------+-----------------------+------------------+
|      200|          0.0|     0.3048|              31.7|             0.00331266|           117.195|
|      200|          0.0|     0.3048|              39.6|             0.00310138|           118.129|
|      200|          7.3|     0.2286|              39.6|              0.0123481|           130.989|
|      200|          7.3|     0.2286|              55.5|              0.0111706|           135.234|
|      200|          7.3|     0.2286|              71.3|              0.0104404|           138.758|
|      200|          8.9|     0.1016|              71.3|              0.0103088|           133.503|
|      200|          9.9|     0.1524|              31.7|              0.0252785|           127.299|


In [129]:
Pipeline_model = pipeline.fit(training_data)

In [131]:
print("Part 2 - Evaluation")
print("Total rows = ", rawcount4)
ps = [str(x).split("_")[0] for x in pipeline.getStages()]

print("Pipeline Stage 1 = ", ps[0])
print("Pipeline Stage 2 = ", ps[1])
print("Pipeline Stage 3 = ", ps[2])

print("Label column = ", lr.getLabelCol())

Part 2 - Evaluation
Total rows =  1499
Pipeline Stage 1 =  VectorAssembler
Pipeline Stage 2 =  StandardScaler
Pipeline Stage 3 =  LinearRegression
Label column =  SoundLevelDecibels


In [133]:
predictions = Pipeline_model.transform(testing_data)

In [134]:
from pyspark.ml.evaluation import RegressionEvaluator

# Define the evaluation metrics
evaluator = RegressionEvaluator(predictionCol="prediction", labelCol="SoundLevelDecibels", metricName="mae")
mae = evaluator.evaluate(predictions)

evaluator.setMetricName("mse")
mse = evaluator.evaluate(predictions)

evaluator.setMetricName("r2")
r2 = evaluator.evaluate(predictions)

print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("R-squared (R^2):", r2)

Mean Absolute Error (MAE): 3.913679095881318
Mean Squared Error (MSE): 24.997666255020178
R-squared (R^2): 0.495968840897543


In [136]:
print("Mean Squared Error = ", round(mse,2))
print("Mean Absolute Error = ", round(mae,2))
print("R Squared = ", round(r2,2))

lrModel = Pipeline_model.stages[-1]

print("Intercept = ", round(lrModel.intercept,2))


Mean Squared Error =  25.0
Mean Absolute Error =  3.91
R Squared =  0.5
Intercept =  124.83


In [137]:
# pipeline_model.save(model_path)
Pipeline_model.save("pipeline_model.model")

In [139]:
from pyspark.ml import PipelineModel
Final_model = PipelineModel.load("pipeline_model.model")

In [141]:
pred = Final_model.transform(testing_data)

In [142]:
pred.select("prediction", "SoundLevelDecibels").show()

+------------------+------------------+
|        prediction|SoundLevelDecibels|
+------------------+------------------+
|122.59722914376378|           128.679|
|127.37968204568342|            133.42|
|130.34077425073457|           119.146|
| 131.1101697511273|           116.074|
|127.12627360125776|           134.319|
| 127.8945637390452|            125.01|
| 131.0622098122442|           125.941|
|125.73739953849169|           130.588|
|121.53249832198978|           128.354|
|124.20059665618763|           121.783|
|125.87997778533499|            122.94|
|125.24362112903022|           116.146|
|126.06429872612159|           114.044|
|127.67830278943427|           109.951|
| 121.2502214756427|           125.974|
|123.31966959831726|           116.066|
|124.20046348885761|           118.595|
|126.16068839642757|           126.395|
| 122.5337859220566|           130.089|
|123.42922049989846|           131.889|
+------------------+------------------+
only showing top 20 rows

